<a href="https://colab.research.google.com/github/JohnSili/NovosibCFP/blob/main/%D1%86%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D0%BE%D0%B9_%D0%BF%D1%80%D0%BE%D1%80%D1%8B%D0%B2_%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D0%B8%D0%B1%D0%B8%D1%80%D1%81%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/bsm_data_train.csv')

In [13]:
import pandas as pd

# Загрузка данных о потоке BSM
bsm_data_train = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/bsm_data_train.csv')

# Загрузка данных о расписании рейсов
flight_rasp_data = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/flight_rasp_data.csv')

# Загрузка данных о прогнозе прибытия
arrival_profile_data = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/arrival_profile_data.csv')
arrival_profile_value = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/arrival_profile_value.csv')
# Объединение по i_id
combined_data = pd.merge(bsm_data_train, flight_rasp_data, on='i_id', how='inner')
combined_data['MessageReceivedDate'] = pd.to_datetime(combined_data['MessageReceivedDate'])
combined_data['MessageProcessedDate'] = pd.to_datetime(combined_data['MessageProcessedDate'])

# Вычисление разницы во времени
#combined_data['TimeDifference'] = combined_data['MessageProcessedDate'] - combined_data['MessageReceivedDate']

# Преобразование TimeDifference в секунды
#combined_data['TimeDifferenceSeconds'] = combined_data['TimeDifference'].dt.total_seconds()

# Расчет объема за минуту
combined_data['VolumePerMinute'] = 60 / (combined_data['MessageProcessedDate'] - combined_data['MessageReceivedDate']).dt.total_seconds()

# Преобразование local_or_transfer в бинарный формат и создание нового столбца
combined_data['local_or_transfer_binary'] = combined_data['local_or_transfer'].map({'L': 1, 'T': 0})
# Удаление столбца 'local_or_transfer' с заменой
combined_data.drop(columns=['local_or_transfer'], inplace=True)
combined_data['ReceivedHour'] = combined_data['MessageReceivedDate'].dt.hour
combined_data['ReceivedMinute'] = combined_data['MessageReceivedDate'].dt.minute

combined_data['departure_terminal_encoded'] = combined_data['departure_terminal'].astype('category').cat.codes.astype(int)
combined_data['checkin_terminal_encoded'] = combined_data['checkin_terminal'].astype('category').cat.codes.astype(int)

combined_data['Weekday of Message'] = combined_data['MessageReceivedDate'].dt.weekday
combined_data['Hour of Message'] = combined_data['MessageReceivedDate'].dt.hour

In [32]:
k = 0
xx = []
for i in range(len(bsm_data_train['i_id'])):
  if bsm_data_train['i_id'].iloc[i] == 8628981.0:
    k += 1
    xx.append(bsm_data_train['pax_id_hash'].iloc[i])
print(k)
print(len(list(set(xx))))

65
57


In [26]:
features = ['departure_terminal_encoded', 'checkin_terminal_encoded', 'Weekday of Message', 'Hour of Message']
target = 'VolumePerMinute'

X = combined_data[features]
y = combined_data[target]

In [27]:
print(min(y))

0.5301758416541487


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Разбиение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализация и обучение модели
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Прогнозирование на тестовом наборе данных
y_pred = model.predict(X_test)

# Оценка модели с использованием MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

KeyboardInterrupt: ignored

Матрица корелляции двух первых файлов
Откинуть ненужные
Сделать пятиминутны интервал, плюс день
Затем добавить третий файл к объединению
и y pax